In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from scipy.stats import norm
import time as ttt
import iisignature as iisig
from tqdm import *
from einops import rearrange
import signatory

In [2]:
import torch
import torch.nn as nn
import pandas as pd 

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

In [3]:
data_type=torch.float32

In [4]:
x0 = 100.0 # initial condition
sigma = 0.15 # volatility
d=100
K=100.0

segs=20
r = 0.05 # risk free rate
batch_size = 100 # batch size
steps=1000
T = 1 # maturity
dt = T/steps # mesh size

sqrt_dt=np.sqrt(dt); 
dt_new = T/segs # new mesh after shrinkage
level = 3 # truncation level logsig 91 sig 258

In [5]:
batch_in=100
MOMENTUM = 0.99
EPSILON = 1e-6
import warnings
warnings.filterwarnings("ignore")

In [6]:
def create_stock(x0,r,sigma,T,steps,n_path,dW):  
    s_vec=[];
    w0=np.ones((n_path,1,d))*1e-6
    dW=np.concatenate((w0,dW),axis=1) ## the first time slot is then 0
    s_vec.append(np.ones((n_path,d))*100.0)
    for i in range(steps): 
        s_vectemp=s_vec[-1]+ r*s_vec[-1]*dt+ dW[:,i+1,:]*s_vec[-1]*sigma
        s_vec.append(s_vectemp)
    BM_path=np.cumsum(dW,axis=1) ## find the cumulative sum
    S_path=rearrange(np.array(s_vec), 'b c h -> c b h') 
    return BM_path, S_path

def jointime(T,path): 
    n_path, steps, d=path.shape
    dt=T/(steps-1); 
    
    times=np.arange(0,T,dt) ## This can be taken out
    times=np.append(times,T); 
    times_vec=np.tile(times,[1,1]); 
    times_vec=np.transpose(times_vec)
    times_vec=np.tile(times_vec,[n_path,1,1])
    times_vec=np.concatenate((times_vec, path),axis=2)
    return times_vec


##  the dimension
$$\frac{d^{N+1}-1}{d-1}$$

In [7]:
embedding=signatory.Augment(in_channels=d+1,layer_sizes=(64,64,5),kernel_size=1,include_original=False,include_time=True)  

In [8]:
def ComputeMultiLevelSig(s_embed, number_of_segment,level):
    n_batch, nsteps, chanels = s_embed.shape
    s_pathclass = signatory.Path(s_embed,level) ## Create a path class 

    t_vec = np.arange(0, nsteps-1, int(nsteps / 20))
    t_vec = np.append(t_vec, nsteps-1)
    MultiLevelSig = []

    ll=torch.zeros(n_batch, 258) #iisig.sig(np.expand_dims(path[:,0,:],axis=1),depth)
    MultiLevelSig.append(ll)
    for i in range(len(t_vec)-1):    
    ## Notice that we only use the signature of the concatenation of time and space.
        MultiLevelSig.append(s_pathclass.signature(0, t_vec[i+1]+1)) ##if not
        #MultiLevelSig.append(path_class.signature(t_vec[i],t_vec[i+1]+1))
    MultiLevelSig=torch.stack(MultiLevelSig)  
    MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
    return MultiLevelSig

In [9]:
### The x_ten here has to be a path
### The level of discretization is on the fine mesh
def terminal_loss(x_ten):
    res=torch.relu(torch.sum(x_ten,dim=(1,2))*dt/d-K)
    return res

In [10]:
def generate_samples(model,batch_in=100):
    """
    Produce the signature, dW for computation, YT the terminal condition
    x_ten, selection the path and location to be used
    """
    dW=np.sqrt(dt)*np.random.normal(size=(batch_in, steps,d))
    
    pth2=create_stock(x0,r,sigma,T,steps,batch_in,dW)
    BM_timePath=jointime(T,pth2[0]); 
    S_timePath=jointime(T,pth2[1]);
    
    ########### embed the long sequence to a lower dimension
    BM_timePath=torch.tensor(BM_timePath,dtype=data_type)
    S_timePath=torch.tensor(S_timePath,dtype=data_type)
    s_embed=model.embedding(S_timePath)
    ###########
    
    
    sigs=ComputeMultiLevelSig(s_embed, 20, 3)
    selection = np.linspace(0,steps, segs+1, dtype = np.int)

    BM_seg=BM_timePath[:,selection,1:]
    dW=BM_seg[:,1:,:]-BM_seg[:,:-1,:]

    dW=torch.tensor(dW,dtype=torch.float32)
    sigs=torch.tensor(sigs,dtype=torch.float32)
    S_timePath=torch.tensor(S_timePath,dtype=torch.float32)
    
    x_ten=S_timePath[:,:,1:]
    
    YT=torch.relu(torch.sum(x_ten,dim=(1,2))*dt/d-K)
    
    return sigs, dW, YT, x_ten, selection[:-1]

In [11]:
class Config(object):
    n_layer = 4
    batch_size = 1024
    valid_size = 1024
    
    
   ## We embed the squence dim=100 into a sequence of dim 5, then plus the time dimension we get 6. 
    dim=258; 
    Ntime=20; 
    delta=1/Ntime
    sqrt_deltaT=np.sqrt(1.0/Ntime); 
    lam=1; 

    logging_frequency = 100
    verbose = True
    y_init_range = [0, 1]
    
    num_hiddens = [dim,64,64,d] ## 256 ,256
    
def get_config(name):
    try:
        return globals()[name]
    except KeyError:
        raise KeyError("config not defined.")

cfg=get_config('Config')

In [12]:
class Dense(nn.Module): 
    def __init__(self,cin, cout, batch_norm=False, activate=True): 
        super(Dense,self).__init__()
        self.cin=cin; 
        self.cout=cout; 
        self.activate=activate; 
        
        self.linear=nn.Linear(self.cin,self.cout) #The linear layer
        #BatchNorm1d: it requires the input to be a correct size
        if batch_norm: 
            self.bn=nn.BatchNorm1d(cout,eps=EPSILON,momentum=MOMENTUM)
        else: 
            self.bn=None
      #  nn.init.normal_(self.linear.weight,std=5.0/np.sqrt(cin+cout))
        # This is the He initialization
        
    def forward(self,x): 
        x=self.linear(x)
        if self.bn is not None:
            x=self.bn(x)
        if self.activate:
            x=torch.tanh(x)
        return x 
    
class FFN(nn.Module):
    def __init__(self, config):
        super(FFN,self).__init__()
        self.config=config
        
        self.bn=nn.BatchNorm1d(config.num_hiddens[0],eps=EPSILON,momentum=MOMENTUM) ## So there is batch norm no problem
        # range(1,5): 1,2,3,4
        self.layers=[Dense(config.num_hiddens[i-1],config.num_hiddens[i]) for i in range(1, len(config.num_hiddens)-1)]
        self.layers+=[Dense(config.num_hiddens[-2], config.num_hiddens[-1],activate=False)]
        self.layers=nn.Sequential(*self.layers)
    
    def forward(self,x):
      #  x=self.bn(x)
        x=self.layers(x)
        return x 
    
class Lookback_PPDE_Backward(nn.Module):
    def __init__(self,cfg): 
        super(Lookback_PPDE_Backward,self).__init__()
        self.cfg=cfg
        self.Ntime=self.cfg.Ntime 
        self.mList=nn.ModuleList([FFN(self.cfg) for _ in range(self.Ntime)])
        self.embedding=signatory.Augment(in_channels=d+1,layer_sizes=(64,64,5),kernel_size=1,include_original=False,include_time=True)  
        
    def forward(self,batch_sig,batch_dW,batch_YT, batch_x, batch_sel): 
        Y=batch_YT
        for i in np.arange(segs-1,-1, -1):
            Y=Y-Y*r*T/segs-sigma*torch.sum( self.mList[i](batch_sig[:,i,:])*batch_dW[:,i,:],axis=1,keepdim=True)
            gi=terminal_loss(batch_x[:,:batch_sel[i],:])
            Y=torch.relu(gi-Y)+Y
        return Y

In [13]:
def loss_var(x):
    temp=torch.var(x)
    return temp

In [14]:
import torch.optim as optim
from torch.nn import Parameter
import math
model_PPDE_bw=Lookback_PPDE_Backward(cfg)
optimizer=optim.Adam(model_PPDE_bw.parameters(),lr=5e-4)
grad_clip=0.1
y0_mean=[];
loss_vec=[];
#scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=80, gamma=0.1)

In [15]:
#ss=generate_samples(model_PPDE_bw)

# dim 100

In [162]:
for i in range(150):
    batch_sig, batch_dw, batch_y,batch_x, batch_sel =generate_samples(model_PPDE_bw,batch_in=500)

    x_temp=model_PPDE_bw(batch_sig,batch_dw,batch_y,batch_x, batch_sel)
    loss_temp=loss_var(x_temp)

    optimizer.zero_grad()
    loss_temp.backward()
    optimizer.step()
#   scheduler.step()
    y0_val=x_temp.mean().cpu().detach().numpy()
    loss_val=loss_temp.cpu().detach().numpy()
    
    y0_mean.append(y0_val)
    loss_vec.append(loss_val)
    if grad_clip: 
        nn.utils.clip_grad_value_(model_PPDE_bw.parameters(), grad_clip)
    
    print("Iter:", i, 'The mean Y0 is', y0_val , 'Variance is:' ,loss_val)

Iter: 0 The mean Y0 is 2.5514688 Variance is: 0.7387655
Iter: 1 The mean Y0 is 2.5324154 Variance is: 0.7212821
Iter: 2 The mean Y0 is 2.559755 Variance is: 0.77636796
Iter: 3 The mean Y0 is 2.4629514 Variance is: 0.7433861
Iter: 4 The mean Y0 is 2.4936957 Variance is: 0.71502787
Iter: 5 The mean Y0 is 2.5134683 Variance is: 0.8749918
Iter: 6 The mean Y0 is 2.50242 Variance is: 0.7255636
Iter: 7 The mean Y0 is 2.5220277 Variance is: 0.8325559
Iter: 8 The mean Y0 is 2.4584365 Variance is: 0.7595968
Iter: 9 The mean Y0 is 2.5187154 Variance is: 0.7009184
Iter: 10 The mean Y0 is 2.491131 Variance is: 0.76011735
Iter: 11 The mean Y0 is 2.5503144 Variance is: 0.75345474
Iter: 12 The mean Y0 is 2.621376 Variance is: 0.83541626
Iter: 13 The mean Y0 is 2.4792657 Variance is: 0.6624706
Iter: 14 The mean Y0 is 2.5393696 Variance is: 0.6895864
Iter: 15 The mean Y0 is 2.5468638 Variance is: 0.7182934
Iter: 16 The mean Y0 is 2.5146716 Variance is: 0.788675
Iter: 17 The mean Y0 is 2.548294 Variance 

Iter: 144 The mean Y0 is 2.549487 Variance is: 0.7914217
Iter: 145 The mean Y0 is 2.5188208 Variance is: 0.76334846
Iter: 146 The mean Y0 is 2.5440335 Variance is: 0.71988076
Iter: 147 The mean Y0 is 2.577703 Variance is: 0.6868734
Iter: 148 The mean Y0 is 2.4506195 Variance is: 0.65571153
Iter: 149 The mean Y0 is 2.577914 Variance is: 0.783023


In [183]:
df=pd.DataFrame()
df['y_pred']=y0_mean
df['loss_var']=loss_vec

df_temp=df[11:]
lw=df_temp.loss_var.mean()-2.0*df_temp.loss_var.std()
up=df_temp.loss_var.mean()+2.0*df_temp.loss_var.std()
df1=df_temp[df_temp.loss_var>=lw]
df1=df1[df1.loss_var<=up]
df1.y_pred.mean()

nan

In [36]:
import time

In [37]:
for epoch in np.arange(32,51,1):
    model_PPDE_bw=Lookback_PPDE_Backward(cfg)
    optimizer=optim.Adam(model_PPDE_bw.parameters(),lr=5e-4)
    grad_clip=0.1
    y0_mean=[];
    loss_vec=[];
    
    start_time = time.time()
    for i in range(100):
        batch_sig, batch_dw, batch_y,batch_x, batch_sel =generate_samples(model_PPDE_bw,batch_in=500)

        x_temp=model_PPDE_bw(batch_sig,batch_dw,batch_y,batch_x, batch_sel)
        loss_temp=loss_var(x_temp)

        optimizer.zero_grad()
        loss_temp.backward()
        optimizer.step()
    #   scheduler.step()
        y0_val=x_temp.mean().cpu().detach().numpy()
        loss_val=loss_temp.cpu().detach().numpy()

        y0_mean.append(y0_val)
        loss_vec.append(loss_val)
        if grad_clip: 
            nn.utils.clip_grad_value_(model_PPDE_bw.parameters(), grad_clip)

        print("Iter:", i, 'The mean Y0 is', y0_val , 'Variance is:' ,loss_val)
    
    print("Epoch:", epoch)
    print("--- %s seconds ---" % (time.time() - start_time))
    
    y_pred=np.array(y0_mean)
    loss_val=np.array(loss_vec)
    iters=np.arange(0,len(y_pred),1)+1
    df=pd.DataFrame()
    df['iter']=iters
    df['y_pred']=y_pred
    df['loss_var']=loss_val
    df.to_csv('eg2_trained_dataDim100/method2'+'_'+str(epoch)+'.csv')

Iter: 0 The mean Y0 is 2.5048232 Variance is: 0.6961558
Iter: 1 The mean Y0 is 2.5464895 Variance is: 0.8162427
Iter: 2 The mean Y0 is 2.4703546 Variance is: 0.7321826
Iter: 3 The mean Y0 is 2.4970312 Variance is: 0.7347728
Iter: 4 The mean Y0 is 2.5249763 Variance is: 0.69477063
Iter: 5 The mean Y0 is 2.547173 Variance is: 0.7101241
Iter: 6 The mean Y0 is 2.5266445 Variance is: 0.7944495
Iter: 7 The mean Y0 is 2.5989323 Variance is: 0.82316625
Iter: 8 The mean Y0 is 2.584921 Variance is: 0.776621
Iter: 9 The mean Y0 is 2.5296638 Variance is: 0.76960367
Iter: 10 The mean Y0 is 2.5119703 Variance is: 0.61657476
Iter: 11 The mean Y0 is 2.5660138 Variance is: 0.72632885
Iter: 12 The mean Y0 is 2.4960694 Variance is: 0.7032644
Iter: 13 The mean Y0 is 2.515199 Variance is: 0.7559194
Iter: 14 The mean Y0 is 2.4797306 Variance is: 0.78891003
Iter: 15 The mean Y0 is 2.5066478 Variance is: 0.74602115
Iter: 16 The mean Y0 is 2.5034592 Variance is: 0.7526983
Iter: 17 The mean Y0 is 2.5266023 Vari

Iter: 44 The mean Y0 is 2.5625098 Variance is: 0.781964
Iter: 45 The mean Y0 is 2.4303157 Variance is: 0.826312
Iter: 46 The mean Y0 is 2.477278 Variance is: 0.651965
Iter: 47 The mean Y0 is 2.4310744 Variance is: 0.66215175
Iter: 48 The mean Y0 is 2.5069778 Variance is: 0.7669341
Iter: 49 The mean Y0 is 2.5208595 Variance is: 0.7719219
Iter: 50 The mean Y0 is 2.523353 Variance is: 0.78318447
Iter: 51 The mean Y0 is 2.5449297 Variance is: 0.75911176
Iter: 52 The mean Y0 is 2.5211577 Variance is: 0.68608546
Iter: 53 The mean Y0 is 2.5296755 Variance is: 0.7507608
Iter: 54 The mean Y0 is 2.476977 Variance is: 0.7297324
Iter: 55 The mean Y0 is 2.562062 Variance is: 0.70976996
Iter: 56 The mean Y0 is 2.5596364 Variance is: 0.7468765
Iter: 57 The mean Y0 is 2.5593233 Variance is: 0.7824853
Iter: 58 The mean Y0 is 2.5018713 Variance is: 0.79780227
Iter: 59 The mean Y0 is 2.5448377 Variance is: 0.77003086
Iter: 60 The mean Y0 is 2.565993 Variance is: 0.7527897
Iter: 61 The mean Y0 is 2.493323

Iter: 88 The mean Y0 is 2.494199 Variance is: 0.75638294
Iter: 89 The mean Y0 is 2.5164526 Variance is: 0.71920896
Iter: 90 The mean Y0 is 2.5002327 Variance is: 0.79361844
Iter: 91 The mean Y0 is 2.4616013 Variance is: 0.7500726
Iter: 92 The mean Y0 is 2.5738268 Variance is: 0.79920214
Iter: 93 The mean Y0 is 2.503671 Variance is: 0.67293197
Iter: 94 The mean Y0 is 2.534034 Variance is: 0.6727593
Iter: 95 The mean Y0 is 2.5242004 Variance is: 0.7924704
Iter: 96 The mean Y0 is 2.4680495 Variance is: 0.6959575
Iter: 97 The mean Y0 is 2.4939573 Variance is: 0.78432757
Iter: 98 The mean Y0 is 2.498712 Variance is: 0.72768086
Iter: 99 The mean Y0 is 2.544153 Variance is: 0.8372007
Epoch: 34
--- 307.0031096935272 seconds ---
Iter: 0 The mean Y0 is 2.5148385 Variance is: 0.7671808
Iter: 1 The mean Y0 is 2.538253 Variance is: 0.7896538
Iter: 2 The mean Y0 is 2.5558033 Variance is: 0.7441394
Iter: 3 The mean Y0 is 2.5232193 Variance is: 0.8378685
Iter: 4 The mean Y0 is 2.5334885 Variance is: 0

Iter: 31 The mean Y0 is 2.5477455 Variance is: 0.6645384
Iter: 32 The mean Y0 is 2.5120997 Variance is: 0.7338274
Iter: 33 The mean Y0 is 2.5263 Variance is: 0.6812843
Iter: 34 The mean Y0 is 2.5260577 Variance is: 0.67733186
Iter: 35 The mean Y0 is 2.4694264 Variance is: 0.7443316
Iter: 36 The mean Y0 is 2.5280135 Variance is: 0.6922447
Iter: 37 The mean Y0 is 2.496331 Variance is: 0.70801723
Iter: 38 The mean Y0 is 2.538534 Variance is: 0.81557876
Iter: 39 The mean Y0 is 2.5103035 Variance is: 0.7702407
Iter: 40 The mean Y0 is 2.495295 Variance is: 0.72587603
Iter: 41 The mean Y0 is 2.5131764 Variance is: 0.70264953
Iter: 42 The mean Y0 is 2.5753787 Variance is: 0.74244076
Iter: 43 The mean Y0 is 2.5175126 Variance is: 0.7884959
Iter: 44 The mean Y0 is 2.5004754 Variance is: 0.7714681
Iter: 45 The mean Y0 is 2.4654136 Variance is: 0.65561384
Iter: 46 The mean Y0 is 2.5615883 Variance is: 0.79474974
Iter: 47 The mean Y0 is 2.5696185 Variance is: 0.71845746
Iter: 48 The mean Y0 is 2.54

Iter: 75 The mean Y0 is 2.517256 Variance is: 0.68038386
Iter: 76 The mean Y0 is 2.497502 Variance is: 0.7394207
Iter: 77 The mean Y0 is 2.4167836 Variance is: 0.7275921
Iter: 78 The mean Y0 is 2.5133765 Variance is: 0.704396
Iter: 79 The mean Y0 is 2.5398157 Variance is: 0.7265753
Iter: 80 The mean Y0 is 2.5078106 Variance is: 0.76927656
Iter: 81 The mean Y0 is 2.5236878 Variance is: 0.71894264
Iter: 82 The mean Y0 is 2.498375 Variance is: 0.69238454
Iter: 83 The mean Y0 is 2.5385406 Variance is: 0.7986415
Iter: 84 The mean Y0 is 2.5316885 Variance is: 0.6630046
Iter: 85 The mean Y0 is 2.5127788 Variance is: 0.6974392
Iter: 86 The mean Y0 is 2.5022235 Variance is: 0.73746926
Iter: 87 The mean Y0 is 2.4612002 Variance is: 0.76034975
Iter: 88 The mean Y0 is 2.5343175 Variance is: 0.7196396
Iter: 89 The mean Y0 is 2.5354629 Variance is: 0.725023
Iter: 90 The mean Y0 is 2.5231693 Variance is: 0.7347892
Iter: 91 The mean Y0 is 2.4458506 Variance is: 0.63668674
Iter: 92 The mean Y0 is 2.451

...

Iter: 90 The mean Y0 is 2.5175002 Variance is: 0.6533022
Iter: 91 The mean Y0 is 2.479436 Variance is: 0.74510926
Iter: 92 The mean Y0 is 2.4777126 Variance is: 0.73952025
Iter: 93 The mean Y0 is 2.5689876 Variance is: 0.82900864
Iter: 94 The mean Y0 is 2.4908323 Variance is: 0.75499755
Iter: 95 The mean Y0 is 2.5329316 Variance is: 0.68034065
Iter: 96 The mean Y0 is 2.5196173 Variance is: 0.78374535
Iter: 97 The mean Y0 is 2.494747 Variance is: 0.7442679
Iter: 98 The mean Y0 is 2.5824375 Variance is: 0.64269525
Iter: 99 The mean Y0 is 2.5492396 Variance is: 0.6358817
Epoch: 44
--- 310.18518924713135 seconds ---
Iter: 0 The mean Y0 is 2.4740481 Variance is: 0.72658026
Iter: 1 The mean Y0 is 2.5759475 Variance is: 0.7474551
Iter: 2 The mean Y0 is 2.561266 Variance is: 0.71452785
Iter: 3 The mean Y0 is 2.5335093 Variance is: 0.7750698
Iter: 4 The mean Y0 is 2.5379758 Variance is: 0.714637
Iter: 5 The mean Y0 is 2.4543145 Variance is: 0.7609188
Iter: 6 The mean Y0 is 2.4911773 Variance is

Iter: 33 The mean Y0 is 2.513476 Variance is: 0.7207187
Iter: 34 The mean Y0 is 2.4879303 Variance is: 0.6380449
Iter: 35 The mean Y0 is 2.4566195 Variance is: 0.6898601
Iter: 36 The mean Y0 is 2.479803 Variance is: 0.69909227
Iter: 37 The mean Y0 is 2.544725 Variance is: 0.79070777
Iter: 38 The mean Y0 is 2.5413458 Variance is: 0.6672695
Iter: 39 The mean Y0 is 2.436579 Variance is: 0.72689456
Iter: 40 The mean Y0 is 2.516082 Variance is: 0.6861213
Iter: 41 The mean Y0 is 2.454553 Variance is: 0.68460804
Iter: 42 The mean Y0 is 2.6227078 Variance is: 0.703186
Iter: 43 The mean Y0 is 2.5052707 Variance is: 0.6700086
Iter: 44 The mean Y0 is 2.5386875 Variance is: 0.700771
Iter: 45 The mean Y0 is 2.4651296 Variance is: 0.70979995
Iter: 46 The mean Y0 is 2.5131767 Variance is: 0.7298737
Iter: 47 The mean Y0 is 2.4623473 Variance is: 0.8203625
Iter: 48 The mean Y0 is 2.4662173 Variance is: 0.68549603
Iter: 49 The mean Y0 is 2.5116444 Variance is: 0.7253562
Iter: 50 The mean Y0 is 2.5016994

Iter: 76 The mean Y0 is 2.539108 Variance is: 0.6794789
Iter: 77 The mean Y0 is 2.509361 Variance is: 0.73011714
Iter: 78 The mean Y0 is 2.461143 Variance is: 0.76384604
Iter: 79 The mean Y0 is 2.5535762 Variance is: 0.74142915
Iter: 80 The mean Y0 is 2.519349 Variance is: 0.83309036
Iter: 81 The mean Y0 is 2.51192 Variance is: 0.7768757
Iter: 82 The mean Y0 is 2.5834525 Variance is: 0.7425009
Iter: 83 The mean Y0 is 2.529474 Variance is: 0.71139544
Iter: 84 The mean Y0 is 2.5098412 Variance is: 0.7723919
Iter: 85 The mean Y0 is 2.4958303 Variance is: 0.80161774
Iter: 86 The mean Y0 is 2.487245 Variance is: 0.68871945
Iter: 87 The mean Y0 is 2.4994411 Variance is: 0.7434234
Iter: 88 The mean Y0 is 2.4381144 Variance is: 0.7673454
Iter: 89 The mean Y0 is 2.502627 Variance is: 0.66509295
Iter: 90 The mean Y0 is 2.5291095 Variance is: 0.73518103
Iter: 91 The mean Y0 is 2.5500762 Variance is: 0.7070407
Iter: 92 The mean Y0 is 2.458844 Variance is: 0.6993876
Iter: 93 The mean Y0 is 2.495361

Iter: 19 The mean Y0 is 2.56743 Variance is: 0.75280195
Iter: 20 The mean Y0 is 2.5118408 Variance is: 0.745828
Iter: 21 The mean Y0 is 2.5594027 Variance is: 0.7231041
Iter: 22 The mean Y0 is 2.432637 Variance is: 0.7672116
Iter: 23 The mean Y0 is 2.5081227 Variance is: 0.79351383
Iter: 24 The mean Y0 is 2.5363195 Variance is: 0.7303326
Iter: 25 The mean Y0 is 2.531469 Variance is: 0.77790844
Iter: 26 The mean Y0 is 2.5133305 Variance is: 0.7410968
Iter: 27 The mean Y0 is 2.477331 Variance is: 0.7264078
Iter: 28 The mean Y0 is 2.540909 Variance is: 0.69497293
Iter: 29 The mean Y0 is 2.462981 Variance is: 0.70090854
Iter: 30 The mean Y0 is 2.5334992 Variance is: 0.7998049
Iter: 31 The mean Y0 is 2.576822 Variance is: 0.6443664
Iter: 32 The mean Y0 is 2.5779288 Variance is: 0.7069831
Iter: 33 The mean Y0 is 2.4878447 Variance is: 0.7325588
Iter: 34 The mean Y0 is 2.4644926 Variance is: 0.72651213
Iter: 35 The mean Y0 is 2.4868858 Variance is: 0.7247748
Iter: 36 The mean Y0 is 2.4985733 

Iter: 62 The mean Y0 is 2.462518 Variance is: 0.7691459
Iter: 63 The mean Y0 is 2.4558814 Variance is: 0.6899689
Iter: 64 The mean Y0 is 2.5812478 Variance is: 0.65854734
Iter: 65 The mean Y0 is 2.524085 Variance is: 0.7221374
Iter: 66 The mean Y0 is 2.499833 Variance is: 0.7142378
Iter: 67 The mean Y0 is 2.4776568 Variance is: 0.70714927
Iter: 68 The mean Y0 is 2.5161648 Variance is: 0.74286985
Iter: 69 The mean Y0 is 2.4133036 Variance is: 0.7320678
Iter: 70 The mean Y0 is 2.4476962 Variance is: 0.69738966
Iter: 71 The mean Y0 is 2.523498 Variance is: 0.7308254
Iter: 72 The mean Y0 is 2.5294878 Variance is: 0.784908
Iter: 73 The mean Y0 is 2.5203683 Variance is: 0.71977365
Iter: 74 The mean Y0 is 2.555967 Variance is: 0.7492045
Iter: 75 The mean Y0 is 2.4980834 Variance is: 0.78504235
Iter: 76 The mean Y0 is 2.4812107 Variance is: 0.7883315
Iter: 77 The mean Y0 is 2.5053844 Variance is: 0.8060115
Iter: 78 The mean Y0 is 2.5985415 Variance is: 0.8198258
Iter: 79 The mean Y0 is 2.52215

In [38]:
import os 
files_dir=os.getcwd()+'\\eg2_trained_dataDim100\\'
file_names=os.listdir(files_dir)
pred_vec=[]

In [39]:
for file in file_names:
    df=pd.read_csv(files_dir+file)
    df_temp=df[41:]
    lw=df_temp.loss_var.mean()-2.0*df_temp.loss_var.std()
    up=df_temp.loss_var.mean()+2.0*df_temp.loss_var.std()
    df1=df_temp[df_temp.loss_var>=lw]
    df1=df1[df1.loss_var<=up]
    
    pred_vec.append(df1.y_pred.mean())

In [40]:
est_vec=np.array(pred_vec)

In [41]:
est_vec.mean()

2.516165237905791

In [42]:
2*est_vec.std()+ est_vec.mean()

2.526115723394989

In [43]:
-2*est_vec.std()+ est_vec.mean()

2.506214752416593